# Anwendung: Standortprobleme


Standortprobleme (*Facility location problems*) sind eine reichhaltige Klasse von Problemen, bei denen gemischt-ganzzahlige lineare Optimierung eingesetzt wird. Es gibt unzählige Varianten von Standortproblemen. Normalerweise werden dabei für mögliche Standorte von Fabriken/Lagern/Stationen/Burrito-Trucks/... folgende binäre Entscheidungen optimiert:

*Soll ich eine Fabrik/Lager/Station/Burrito-Truck/... an Position $i$ bauen/öffnen (ja/nein)?*

Die Zielfunktion und Nebenbedingungen unterscheiden sich dabei jeweils

Wir stellen hier exemplarische einige vor, bei denen wir auf Modellierungstricks der vorherigen Kapitel zurückgreifen können.

## Transportproblem mit Fixkosten
Wir schauen uns noch einmal das Transportproblem aus Übung 2 an, der eine Brauerei kostenminimal Bier von drei Produktionsstandorten zu fünf
Großkunden transportieren möche. In dem Modell fielen nur variable Kosten für den Transport an. Bei dieser Erweiterung des Transportmodells geht es nun darum, auch Fixkosten zu modellieren, die anfallen, sobald eine Fabrik überhaupt produziert.

Die maximal lieferbare Menge pro Produktionsstandort sowie die Fixkosten sind wie folgt:

| Produktionsstandort   | Kapazität                     | Fixkosten
|---                    |---                            |---
| Oettingen             | 1000                          | 2000
| Mönchengladbach       | 1000                          | 3000
| Braunschweig          | 1000                          | 4000

Die Bedarfe der Kunden sind gegeben durch
| Kunde     | Bedarf    |
|---        |---        |
| Kunde 1   | 270       |
| Kunde 2   | 270       |
| Kunde 3   | 250       |
| Kunde 4   | 160       |
| Kunde 5   | 180       |

Es fallen Kosten für den Transport pro Einheit an, abhängig von Start und Ziel. Diese sind
|                       | Kunde 1   | Kunde 2   | Kunde 3   | Kunde 4   | Kunde 5   |
|---                    |---        |---        |---        |---        |---        |
| Oettingen             | 4         | 5         | 6         | 8         | 10        |
| Mönchengladbach       | 6         | 4         | 3         | 5         | 8         |
| Braunschweig          | 9         | 7         | 4         | 2         | 4         |

Wir nehmen unser bewährtes Vorgehen, um das Modell zu lösen:
1. Indexmengen und Problemdaten
2. Variablen
3. Zielfunktion
4. Constraints

### Indexmengen und Problemdaten
Wir definieren zunächst Indexmengen und die Produktionskapazitäten, Fixkosten, Bedarfe und Transportkosten.

In [111]:
import pandas as pd

standorte = ["Oettingen","Mönchengladbach","Braunschweig"]
kunden = [f"Kunde {i}" for i in range(1,6)]

produktion = pd.DataFrame({"Kapazität":[1000.,1000,1000], 
                           "Fixkosten":[2000.,3000,4000]}, 
                           index=standorte)

bedarfe = pd.DataFrame({"Bedarf":[270.,270,250,160,180]},
                       index=kunden)

kosten = pd.DataFrame.from_records([(4.,5.,6.,8.,10.),(6,4,3,5,8),(9,7,4,2,4)], 
                          index=standorte, columns=kunden)

### Entscheidungsvariablen
Als Entscheidungsvariablen definieren wir
1. Entscheidung, ob an einem Standort produziert werden soll:

    $$
    x_i=\left\{\begin{array}{rl}
        1 & \text{, produziere an Standort }i \\
        0 & \text{, produziere nicht an Standort }i
        \end{array}\right.,\quad \forall i \in \text{Standorte}
    $$

2. Menge an Bier, das von Standort $i$ zu Kunde $j$ transportiert wird:

    $$
    flow_{i,j}\in\R, \quad \forall i\in \text{Standorte}, \forall j\in \text{Kunden}
    $$

In [112]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("Transportproblem mit Fixkosten")

x = m.addVars(standorte, vtype=GRB.BINARY)
flow = m.addVars(standorte, kunden, vtype=GRB.CONTINUOUS)


### Zielfunktion
Ziel ist es, die Kosten zu minimieren. Diese setzen sich zusammen auf den Fixkosten, die anfallen, wenn der Standort produziert, also $x_i=1$ ist und den variablen Kosten, die je nach Menge anfallen.

In [113]:
m.setObjective(gp.quicksum(x[i]*produktion.loc[i,"Fixkosten"] for i in standorte) + 
               gp.quicksum(flow[i,j]*kosten.loc[i,j] for i in standorte for j in kunden), GRB.MINIMIZE)

### Constraints

Wir haben folgende Constraints:
1. Die Produktionskapazitäten müssen eingehalten werden. Wenn $x_i=0$, darf nichts produziert werden.

    $$
    \sum_{j\in\text{Kunden}} flow_{i,j}\leq x_i\cdot \text{Kapazität}_i,\quad \forall i\in\text{Standorte}
    $$

2. Die Bedarfe müssen gedeckt sein:

    $$
    \sum_{i\in\text{Standorte}} flow_{i,j}= \text{Bedarf}_j,\quad \forall j\in\text{Kunden}
    $$

Danach lösen wir das Modell.

In [114]:
kapa_constr = m.addConstrs((gp.quicksum(flow[i,j] for j in kunden) <= x[i]*produktion.loc[i,"Kapazität"] for i in standorte))
bedarf_constr = m.addConstrs((gp.quicksum(flow[i,j] for i in standorte) == bedarfe.loc[j] for j in kunden))

/tmp/ipykernel_185811/1360454537.py:2: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



In [115]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 18 columns and 33 nonzeros
Model fingerprint: 0x449f6b05
Variable types: 15 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 3e+02]
Presolve time: 0.00s
Presolved: 8 rows, 18 columns, 33 nonzeros
Variable types: 15 continuous, 3 integer (3 binary)
Found heuristic solution: objective 12950.000000

Root relaxation: objective 7.410000e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7410.00000    0    3 12950.00

In [116]:
product_flow = []
for i in standorte:
    for j in kunden:
        if flow[i,j].x > 1e-6:
            product_flow.append({"From": i, "To": j, "Flow": flow[i,j].x})
product_flow = pd.DataFrame(product_flow)
product_flow

,From,To,Flow
0,Oettingen,Kunde 1,270.0
1,Mönchengladbach,Kunde 2,270.0
2,Mönchengladbach,Kunde 3,250.0
3,Mönchengladbach,Kunde 4,160.0
4,Mönchengladbach,Kunde 5,180.0


Wir sehen, dass es insgesamt günstiger ist, einen Produktionsstandort (Braunschweig) nicht zu verwenden, um die Fixkosten zu sparen.

## Möglichst weit verteilte Standorte ($p$-Dispersionsproblem)
Ein Unternehmen plant, 5 Geschäfte in einer Region zu eröffnen. Um die Selbstkonkurrenz zu minimieren, möchte es unter gegebenen möglichen Standorten diejenigen auswählen, bei denen die kleinste Distanz zwischen zwei beliebigen Geschäften maximal wird. Es handelt sich also um ein $\max \min$ Problem, allerdings ist die Menge, über die das Minimum gebildet wird -- die Standortpaare -- noch nicht bekannt.

### Indexmengen und Problemdaten
Wir erzeugen und zufällig 25 Standorte von denen der Optimierer 5 auswählen soll, so dass die kleinste Distanz möglichst groß ist (die also möglichst "weit verteilt" sind). Die Distanzen zu den Koordinaten erzeugen wir automatisch mit dem Befehl `distance_matrix` aus dem `scipy.spatial` Paket. Die Distanz zwischen Standort $i$ und Standort $j$ bezeichnen wir mit $d_{ij}$. Die Indexmenge $\text{Standorte}$ enthält hier einfach die Indizes der Standorte.

In [173]:
import numpy as np
from scipy.spatial import distance_matrix
import plotly.express as px

np.random.seed(42)

# Anzahl möglicher Standorte
N = 25
p = 5

# Zufällige Koordinaten und euklidische Distanzen
koordinaten = pd.DataFrame(np.random.random((N, 2)), columns=["x","y"])
distanzen = pd.DataFrame(distance_matrix(koordinaten, koordinaten))
standorte = range(N)

# Visualisieren der möglichen Standorte
px.scatter(data_frame=koordinaten, x="x", y="y", width=700, height=700)

### Entscheidungen
Wie oben soll die Entscheidung optimiert werden, ob an einem Standort ein Geschäft eröffnet werden soll:

$$
    x_i=\left\{\begin{array}{rl}
        1 & \text{, eröffne Geschäft an Standort }i \\
        0 & \text{, sonst.}
        \end{array}\right.,\quad \forall i \in \text{Standorte}
$$

In [169]:
m = gp.Model("p-Dispersion Problem")
x = m.addVars(standorte, vtype=GRB.BINARY)
D = m.addVar(vtype=GRB.CONTINUOUS)

### Zielfunktion
Es soll die kleinste Distanz zwischen den Geschäften maximiert werden. Diese bezeichnen wir mit $D$.

In [170]:
m.setObjective(D, GRB.MAXIMIZE)

### Constraints
1. Es sollen genau $p=5$ Geschäfte gebaut werden: $\sum_{i\in\text{Standorte}}x_i=5$.
2. Wir müssen sicherstellen, dass $D$ auch wirklich die kleinste Distanz zwischen den Geschäften darstellt. Dazu erstellen wir eine Constraint für jedes Paar von möglichen Standorten, so dass $D$ kleiner ist als die Distanz zwischen Standort $i$ und Standort $j$.  Diese Constraints sollen aber nur gelten, wenn an den zwei betreffenden Standorten auch wirklich Geschäfte gebaut werden. Wir schalten dazu alle anderen Constraints mittels einem großen $M$ ab. $M$ kann hier als der maximale Eintrag der Distanzmatrix gewählt werden. Formal:

    $$
    D\leq d_{ij} + M(1-x_i)+M(1-x_j),\quad \forall i\in\text{Standorte}, j\in\text{Standorte}, i<j
    $$
    
   Sobald entweder Standort $i$ oder $j$ nicht bebaut sind, spielt die Distanz $d_{ij}$ für unser Problem keine Rolle. $D$ wird dann durch das Paar $i,j$ nicht eingeschränkt.

In [171]:
m.addConstr(gp.quicksum(x[i] for i in standorte) == p)

M = distanzen.max().max()
for i in standorte:
    for j in standorte:
        if j > i:
            m.addConstr(D <= distanzen.loc[i,j] + M*(1-x[i]) + M*(1-x[j]))    
            
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 301 rows, 26 columns and 925 nonzeros
Model fingerprint: 0x3099525b
Variable types: 1 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+00]
Found heuristic solution: objective 0.1845266
Presolve time: 0.00s
Presolved: 301 rows, 26 columns, 925 nonzeros
Variable types: 1 continuous, 25 integer (25 binary)

Root relaxation: objective 1.913363e+00, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.91336    0   25   

In [172]:
koordinaten["Geschäft"] = "ja"
for i in standorte:
    if x[i].x < 1e-6:
        koordinaten.loc[i, "Geschäft"] = "nein"
px.scatter(data_frame=koordinaten, x="x", y="y", color="Geschäft", width=700, height=700)

## Burrito Optimization Game
Das [Gurobi Burrito Optimization Game](https://burrito.gurobi.com/) ist unter der Haube ein Standortproblem, genauer gesagt eine Variante des [Uncapacitated Facility Location Problems](https://en.wikipedia.org/wiki/Optimal_facility_location#Uncapacitated_facility_location), einem klassischen Problem der linearen Optimierung. Im folgenden beschreiben wir die mathematische Formulierung des Problems.

### Problemdaten und Indexmengen
- $j\in J$: Mögliche Standorte der Trucks
- $i\in I$: Standorte der potentiellen Kunden
- $d_i$: Bedarf der Kunden
- $\alpha_{ij}\in [0,1]$: Anteil des Bedarfs von Kunde $i$, der durch einen Truck am Standort $j$ gedeckt werden kann. Diese Zahl ist abhängig von der Entfernung zwischen $i$ und $j$. Wenn $i$ und $j$ nahe beieinander liegen, ist der Wert nahe 1, es kann also der gesamte Bedarf gedeckt werden. Wenn $i$ und $j$ weit voneinander entfernt liegen, ist der Wert $0$, d.h. der Truck an Standort $j$ wird von den Kunden an Standort $i$ nicht genutzt.
- $r:$ Umsatz pro Burrito
- $k:$ Kosten für die Zutaten
- $f:$ Kosten für den Truck 

### Entscheidungen

$$
    x_j=\left\{\begin{array}{rl}
        1 & \text{, eröffne Truck an Standort }j \\
        0 & \text{, sonst.}
        \end{array}\right.,\quad \forall j \in J
$$

$$
    y_{ij}=\left\{\begin{array}{rl}
        1 & \text{, Kunde }i\text{ wird von Truck }j\text{ bedient.} \\
        0 & \text{, sonst.}
        \end{array}\right.,\quad \forall i\in  I, j \in J
$$

### Zielfunktion
Der Profit soll maximiert werden. Dieser ist gegeben durch den erzielten Gewinn pro Burrito (Umsatz minus Kosten für Zutaten) multipliziert mit dem gedeckten Bedarf der Kunden. Weiterhin müssen die Fixkosten für jeden eröffneten Truck abgezogen werden:

$$
\min \sum_{i\in I,j\in J}(r-k)\alpha_{ij}d_iy_{ij}-\sum_{j\in J}fx_j
$$

### Constraints
1. Jeder Kunde wird von höchstens einem Truck versorgt.

    $$
    \sum_{j\in J}y_{ij} \leq  1 \quad \forall i\in I
    $$

2. Wenn ein Kunde an Standort $i$ durch einen Truck an Standort $j$ versorgt wird, so muss der Truck auch an Standort $j$ eröffnet werden.

    $$
    y_{ij}\leq x_j,\quad \forall i\in I, j\in J
    $$

### Aufgabe

Öffnen Sie das [Spiel]((https://burrito.gurobi.com/)) und versuchen sie die jeweiligen "Problemdaten" im Spiel zu finden. Wo im Spiel kann man $d_i$, $\alpha_{ij}$, $r$, $k$ und $f$ ablesen?